### Campo elegido para el proyecto: Agricultura

Asistente para analizar artículos de agrícultura

In [5]:
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
from typing import List, Dict, Any
from pathlib import Path
from datetime import datetime
import json
import os

from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

try:
    from langchain_text_splitters import RecursiveCharacterTextSplitter
except ImportError:
    try:
        from langchain.text_splitter import RecursiveCharacterTextSplitter
    except ImportError:
        from langchain_core.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import PyPDFLoader, TextLoader, UnstructuredHTMLLoader
from langchain_core.documents import Document

from rich.console import Console
from rich.panel import Panel
from rich.tree import Tree

load_dotenv()

console = Console()


class LocalEmbeddings:
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        console.print(f"[yellow]Cargando modelo de embeddings: {model_name}...[/yellow]")
        self.model = SentenceTransformer(model_name)
        console.print("[green]✓ Modelo de embeddings cargado[/green]")

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        embeddings = self.model.encode(texts, show_progress_bar=True, convert_to_numpy=True, batch_size=32)
        return embeddings.tolist()

    def embed_query(self, text: str) -> List[float]:
        embedding = self.model.encode([text], convert_to_numpy=True)
        return embedding[0].tolist()

    def __call__(self, text: str) -> List[float]:
        return self.embed_query(text)


class TraceabilityLogger:
    def __init__(self):
        self.trace_log = []
        self.start_time = None

    def start_trace(self, query: str):
        self.start_time = datetime.now()
        self.trace_log = [{
            "timestamp": self.start_time.isoformat(),
            "event": "query_received",
            "query": query
        }]

    def log_event(self, agent: str, action: str, details: Dict[str, Any]):
        self.trace_log.append({
            "timestamp": datetime.now().isoformat(),
            "agent": agent,
            "action": action,
            "details": details
        })

    def get_trace(self) -> List[Dict]:
        return self.trace_log

    def print_trace(self):
        tree = Tree("🔍 [bold blue]Trazabilidad del Sistema[/bold blue]")
        for entry in self.trace_log:
            if entry.get("event") == "query_received":
                tree.add(f"[green]Query recibida:[/green] {entry['query']}")
            else:
                agent_node = tree.add(f"[yellow]{entry['agent']}[/yellow]")
                agent_node.add(f"[cyan]Acción:[/cyan] {entry['action']}")
                if entry.get("details"):
                    details_str = json.dumps(entry["details"], indent=2, ensure_ascii=False)
                    agent_node.add(f"[magenta]Detalles:[/magenta]\n{details_str}")
        console.print(tree)


class DocumentIndexer:
    def __init__(self, embeddings_model, trace_logger: TraceabilityLogger):
        self.embeddings = embeddings_model
        self.trace_logger = trace_logger
        self.vector_store = None
        self.documents = []

    def load_documents(self, directory_path: str) -> List[Document]:
        console.print(f"[bold green]📂 Cargando documentos desde: {directory_path}[/bold green]")
        documents = []
        path = Path(directory_path)
        loaders_map = {
            '.pdf': PyPDFLoader,
            '.txt': TextLoader,
            '.html': UnstructuredHTMLLoader,
            '.htm': UnstructuredHTMLLoader
        }
        file_count = 0
        for file_path in path.rglob('*'):
            if file_path.is_file() and file_path.suffix.lower() in loaders_map:
                try:
                    loader_class = loaders_map[file_path.suffix.lower()]
                    if file_path.suffix.lower() == '.txt':
                        loader = loader_class(str(file_path), encoding='utf-8')
                    else:
                        loader = loader_class(str(file_path))
                    docs = loader.load()
                    documents.extend(docs)
                    file_count += 1
                    if file_count % 10 == 0:
                        console.print(f"  ✓ Cargados {file_count} documentos...")
                except Exception as e:
                    console.print(f"[red]  ✗ Error al cargar {file_path.name}: {str(e)}[/red]")
                    continue
        console.print(f"[bold green]✓ Total de documentos cargados: {file_count}[/bold green]")
        self.trace_logger.log_event(agent="DocumentIndexer", action="load_documents", details={"total_documents": file_count, "directory": directory_path})
        self.documents = documents
        return documents

    def process_and_index(self, documents: List[Document], chunk_size: int = 1000, chunk_overlap: int = 200):
        console.print("[bold yellow]🔨 Procesando documentos...[/bold yellow]")
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len, separators=["\n\n", "\n", " ", ""])
        chunks = text_splitter.split_documents(documents)
        console.print(f"  ✓ Documentos divididos en {len(chunks)} chunks")
        console.print("[bold yellow]🧮 Generando embeddings e indexando en FAISS...[/bold yellow]")
        self.vector_store = FAISS.from_documents(chunks, self.embeddings)
        console.print("[bold green]✓ Indexación completada[/bold green]")
        self.trace_logger.log_event(agent="DocumentIndexer", action="process_and_index", details={"total_chunks": len(chunks), "chunk_size": chunk_size, "chunk_overlap": chunk_overlap})
        return self.vector_store

    def save_index(self, path: str = "faiss_index"):
        if self.vector_store:
            self.vector_store.save_local(path)
            console.print(f"[green]✓ Índice guardado en: {path}[/green]")

    def load_index(self, path: str = "faiss_index"):
        try:
            self.vector_store = FAISS.load_local(path, self.embeddings, allow_dangerous_deserialization=True)
            console.print(f"[green]✓ Índice cargado desde: {path}[/green]")
            return self.vector_store
        except Exception as e:
            console.print(f"[red]✗ Error al cargar índice: {str(e)}[/red]")
            return None


class QueryClassifier:
    def __init__(self, trace_logger: TraceabilityLogger):
        self.trace_logger = trace_logger
        self.llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.1, groq_api_key=os.getenv("GROQ_API_KEY"))

    def classify(self, query: str) -> Dict[str, Any]:
        classification_prompt = ChatPromptTemplate.from_messages([
            ("system", """Eres un clasificador de consultas experto. Debes clasificar la intención del usuario en una de estas categorías:

1. "busqueda": El usuario solicita información específica, hechos o datos contenidos en documentos
2. "resumen": El usuario quiere un resumen de uno o varios documentos
3. "comparacion": El usuario quiere comparar o contrastar información de diferentes documentos
4. "general": El usuario hace una pregunta general que no requiere buscar en documentos específicos

Ejemplos:
- "¿Cuál es la capital de Francia?" -> general
- "¿Qué dice el documento sobre las causas de la guerra?" -> busqueda
- "Resume el contenido del artículo sobre IA" -> resumen
- "Compara las conclusiones de estos dos estudios" -> comparacion
- "Explícame qué es la fotosíntesis" -> general
- "¿Qué información hay sobre cambio climático?" -> busqueda

Responde SOLO con un JSON en este formato exacto:
{{"intencion": "busqueda|resumen|comparacion|general", "confianza": 0.95, "razonamiento": "breve explicación"}}"""),
            ("user", "{query}")
        ])
        chain = classification_prompt | self.llm | StrOutputParser()
        response = chain.invoke({"query": query})
        try:
            response_clean = response.strip()
            if response_clean.startswith("```json"):
                response_clean = response_clean[7:]
            if response_clean.endswith("```"):
                response_clean = response_clean[:-3]
            result = json.loads(response_clean.strip())
            self.trace_logger.log_event(agent="QueryClassifier", action="classify_intent", details={"query": query, "clasificacion": result})
            console.print(Panel(f"[bold cyan]Intención:[/bold cyan] {result['intencion']}\n[bold cyan]Confianza:[/bold cyan] {result['confianza']}\n[bold cyan]Razonamiento:[/bold cyan] {result['razonamiento']}", title="🎯 Clasificación de Query"))
            return result
        except json.JSONDecodeError:
            console.print("[red]Error al parsear clasificación, usando 'busqueda' por defecto[/red]")
            return {"intencion": "busqueda", "confianza": 0.5, "razonamiento": "Error en clasificación, asumiendo búsqueda"}


class SemanticRetriever:
    def __init__(self, vector_store: FAISS, trace_logger: TraceabilityLogger):
        self.vector_store = vector_store
        self.trace_logger = trace_logger
        self.llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0, groq_api_key=os.getenv("GROQ_API_KEY"))

    def retrieve(self, query: str, k: int = 5) -> List[Document]:
        console.print(f"[bold yellow]🔎 Buscando documentos relevantes (top-{k})...[/bold yellow]")
        relevant_docs = self.vector_store.similarity_search(query, k=k)
        self.trace_logger.log_event(agent="SemanticRetriever", action="retrieve_documents", details={"query": query, "num_docs_retrieved": len(relevant_docs), "doc_sources": [doc.metadata.get('source', 'unknown') for doc in relevant_docs]})
        console.print(f"[green]✓ Recuperados {len(relevant_docs)} documentos relevantes[/green]")
        return relevant_docs


class RAGResponseGenerator:
    def __init__(self, trace_logger: TraceabilityLogger):
        self.trace_logger = trace_logger
        self.llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.3, groq_api_key=os.getenv("GROQ_API_KEY"))

    def generate_response(self, query: str, context_docs: List[Document], intent: str) -> str:
        console.print("[bold yellow]✍️ Generando respuesta con RAG...[/bold yellow]")
        context = "\n\n".join([f"--- Documento {i+1} (Fuente: {doc.metadata.get('source', 'desconocida')}) ---\n{doc.page_content}" for i, doc in enumerate(context_docs)])
        if intent == "resumen":
            system_prompt = """Eres un asistente experto en resumir documentos. 
            Genera un resumen claro y conciso del contenido proporcionado.
            Incluye las ideas principales y cita las fuentes cuando sea apropiado."""
        elif intent == "comparacion":
            system_prompt = """Eres un asistente experto en análisis comparativo.
            Compara y contrasta la información de los documentos proporcionados.
            Identifica similitudes, diferencias y cita las fuentes de cada afirmación."""
        else:
            system_prompt = """Eres un asistente experto en responder preguntas basándote en documentos.
            Responde la pregunta del usuario usando ÚNICAMENTE la información del contexto proporcionado.
            IMPORTANTE: Cita las fuentes de donde obtienes la información.
            Si la información no está en el contexto, indícalo claramente."""
        rag_prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            ("user", """Contexto de los documentos:
{context}

Pregunta del usuario: {query}

Responde de forma clara, precisa y justificada con citas a las fuentes.""")
        ])
        chain = rag_prompt | self.llm | StrOutputParser()
        response = chain.invoke({"context": context, "query": query})
        self.trace_logger.log_event(agent="RAGResponseGenerator", action="generate_response", details={"query": query, "intent": intent, "num_context_docs": len(context_docs), "response_length": len(response)})
        console.print("[green]✓ Respuesta generada[/green]")
        return response


class ResponseVerifier:
    def __init__(self, trace_logger: TraceabilityLogger):
        self.trace_logger = trace_logger
        self.llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.1, groq_api_key=os.getenv("GROQ_API_KEY"))

    def verify(self, query: str, response: str, context_docs: List[Document]) -> Dict[str, Any]:
        console.print("[bold yellow]✅ Verificando respuesta...[/bold yellow]")
        context = "\n\n".join([doc.page_content for doc in context_docs])
        verification_prompt = ChatPromptTemplate.from_messages([
            ("system", """Eres un evaluador crítico de respuestas. Tu trabajo es verificar:

1. ¿La respuesta está respaldada por el contexto proporcionado?
2. ¿La respuesta es coherente y lógica?
3. ¿Hay signos de alucinación (información inventada no presente en el contexto)?
4. ¿La respuesta responde adecuadamente a la pregunta del usuario?

Responde con un JSON en este formato:
{{
    "es_valida": true/false,
    "confianza": 0.0-1.0,
    "problemas_detectados": ["lista de problemas si existen"],
    "sugerencias_mejora": ["sugerencias si es_valida es false"],
    "razonamiento": "explicación de tu evaluación"
}}"""),
            ("user", """Contexto:
{context}

Pregunta del usuario: {query}

Respuesta a evaluar: {response}

Evalúa la respuesta.""")
        ])
        chain = verification_prompt | self.llm | StrOutputParser()
        verification_result = chain.invoke({"context": context, "query": query, "response": response})
        try:
            verification_clean = verification_result.strip()
            if verification_clean.startswith("```json"):
                verification_clean = verification_clean[7:]
            if verification_clean.endswith("```"):
                verification_clean = verification_clean[:-3]
            result = json.loads(verification_clean.strip())
            self.trace_logger.log_event(agent="ResponseVerifier", action="verify_response", details=result)
            if result["es_valida"]:
                console.print("[bold green]✓ Respuesta verificada exitosamente[/bold green]")
            else:
                console.print("[bold red]✗ Respuesta requiere mejoras[/bold red]")
                console.print(f"[yellow]Problemas: {', '.join(result['problemas_detectados'])}[/yellow]")
            return result
        except json.JSONDecodeError:
            console.print("[red]Error al parsear verificación[/red]")
            return {"es_valida": True, "confianza": 0.5, "problemas_detectados": [], "sugerencias_mejora": [], "razonamiento": "Error en verificación, aceptando respuesta"}


class Orchestrator:
    def __init__(self, indexer: DocumentIndexer, classifier: QueryClassifier, retriever: SemanticRetriever, generator: RAGResponseGenerator, verifier: ResponseVerifier, trace_logger: TraceabilityLogger):
        self.indexer = indexer
        self.classifier = classifier
        self.retriever = retriever
        self.generator = generator
        self.verifier = verifier
        self.trace_logger = trace_logger
        self.llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0, groq_api_key=os.getenv("GROQ_API_KEY"))

    def process_query(self, query: str, max_retries: int = 2) -> Dict[str, Any]:
        console.print(Panel(f"[bold white]{query}[/bold white]", title="📝 Query del Usuario", border_style="blue"))
        self.trace_logger.start_trace(query)
        classification = self.classifier.classify(query)
        intent = classification["intencion"]
        self.trace_logger.log_event(agent="Orchestrator", action="route_query", details={"intent": intent})
        if intent == "general":
            return self._handle_general_query(query)
        else:
            return self._handle_rag_query(query, intent, max_retries)

    def _handle_general_query(self, query: str) -> Dict[str, Any]:
        console.print("[bold cyan]💬 Procesando consulta general (sin RAG)...[/bold cyan]")
        llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.7, groq_api_key=os.getenv("GROQ_API_KEY"))
        general_prompt = ChatPromptTemplate.from_messages([("system", "Eres un asistente útil y conocedor. Responde las preguntas del usuario de manera clara y precisa."), ("user", "{query}")])
        chain = general_prompt | llm | StrOutputParser()
        response = chain.invoke({"query": query})
        self.trace_logger.log_event(agent="Orchestrator", action="handle_general_query", details={"response_length": len(response)})
        return {"query": query, "intent": "general", "response": response, "documents_used": [], "verification": {"es_valida": True, "confianza": 1.0}, "trace": self.trace_logger.get_trace()}

    def _handle_rag_query(self, query: str, intent: str, max_retries: int) -> Dict[str, Any]:
        console.print(f"[bold cyan]🔄 Procesando consulta con RAG (intención: {intent})...[/bold cyan]")
        k = 5 if intent == "busqueda" else 10
        relevant_docs = self.retriever.retrieve(query, k=k)
        if not relevant_docs:
            return {"query": query, "intent": intent, "response": "No se encontraron documentos relevantes para responder a tu consulta.", "documents_used": [], "verification": {"es_valida": False, "confianza": 0.0}, "trace": self.trace_logger.get_trace()}
        for attempt in range(max_retries + 1):
            console.print(f"[cyan]Intento {attempt + 1}/{max_retries + 1} de generación...[/cyan]")
            response = self.generator.generate_response(query, relevant_docs, intent)
            verification = self.verifier.verify(query, response, relevant_docs)
            if verification["es_valida"] or attempt == max_retries:
                return {"query": query, "intent": intent, "response": response, "documents_used": [doc.metadata.get('source', 'unknown') for doc in relevant_docs], "verification": verification, "attempts": attempt + 1, "trace": self.trace_logger.get_trace()}
            console.print("[yellow]⚠️ Respuesta no válida, regenerando...[/yellow]")
        return {"query": query, "intent": intent, "response": response, "documents_used": [doc.metadata.get('source', 'unknown') for doc in relevant_docs], "verification": verification, "trace": self.trace_logger.get_trace()}


def main():
    console.print(Panel("[bold cyan]Sistema Agentic AI Multi-Agente[/bold cyan]\n[white]Universidad Nacional de Colombia - PLN[/white]\n[yellow]Procesamiento y Análisis Inteligente de Documentos[/yellow]", title="🤖 Bienvenido", border_style="green"))
    if not os.getenv("GROQ_API_KEY"):
        console.print("[bold red]Error: GROQ_API_KEY no configurada[/bold red]")
        console.print("[yellow]Por favor, añade tu API key de Groq al archivo .env[/yellow]")
        console.print("[yellow]Obténla en: https://console.groq.com/keys[/yellow]")
        return
    console.print("\n[bold cyan]Inicializando sistema...[/bold cyan]")
    embeddings = LocalEmbeddings(model_name="all-MiniLM-L6-v2")
    trace_logger = TraceabilityLogger()
    indexer = DocumentIndexer(embeddings, trace_logger)
    documents_path = os.getenv("DOCUMENTS_PATH", "./data")
    index_path = os.getenv("FAISS_INDEX_PATH", "./faiss_index")
    documents_path = os.path.abspath(documents_path)
    index_path = os.path.abspath(index_path)
    console.print(f"[cyan]Ruta de documentos: {documents_path}[/cyan]")
    console.print(f"[cyan]Ruta de índice: {index_path}[/cyan]")
    vector_store = indexer.load_index(index_path)
    if vector_store is None:
        if not os.path.exists(documents_path):
            console.print(f"[bold red]Error: No se encuentra el directorio de documentos:[/bold red]")
            console.print(f"[yellow]{documents_path}[/yellow]")
            console.print(f"[cyan]Crea la carpeta 'data' en el directorio del proyecto y coloca tus documentos allí[/cyan]")
            return
        documents = indexer.load_documents(documents_path)
        if not documents:
            console.print("[bold red]No se encontraron documentos para procesar[/bold red]")
            console.print("[yellow]Formatos soportados: PDF, TXT, HTML[/yellow]")
            return
        vector_store = indexer.process_and_index(documents)
        indexer.save_index(index_path)
    classifier = QueryClassifier(trace_logger)
    retriever = SemanticRetriever(vector_store, trace_logger)
    generator = RAGResponseGenerator(trace_logger)
    verifier = ResponseVerifier(trace_logger)
    orchestrator = Orchestrator(indexer, classifier, retriever, generator, verifier, trace_logger)
    console.print("[bold green]✓ Sistema inicializado exitosamente[/bold green]\n")
    while True:
        console.print("\n" + "=" * 80)
        query = console.input("[bold cyan]👤 Ingresa tu consulta (o 'salir' para terminar): [/bold cyan]")
        if query.lower() in ['salir', 'exit', 'quit']:
            console.print("[bold green]¡Hasta luego![/bold green]")
            break
        if not query.strip():
            continue
        result = orchestrator.process_query(query)
        console.print("\n")
        console.print(Panel(f"[bold white]{result['response']}[/bold white]", title="🤖 Respuesta del Sistema", border_style="green"))
        if result.get("documents_used"):
            console.print(f"\n[cyan]📚 Documentos utilizados: {len(result['documents_used'])}[/cyan]")
        console.print(f"[cyan]🎯 Intención detectada: {result['intent']}[/cyan]")
        if result.get("verification"):
            ver = result["verification"]
            console.print(f"[cyan]✅ Confianza de verificación: {ver.get('confianza', 0):.2f}[/cyan]")
        if console.input("\n[yellow]¿Mostrar trazabilidad completa? (s/n): [/yellow]").lower() == 's':
            trace_logger.print_trace()


if __name__ == "__main__":
    main()


╭───────────────────────────────────────────────── 🤖 Bienvenido ─────────────────────────────────────────────────╮
│ Sistema Agentic AI Multi-Agente                                                                                 │
│ Universidad Nacional de Colombia - PLN                                                                          │
│ Procesamiento y Análisis Inteligente de Documentos                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Inicializando sistema...

Cargando modelo de embeddings: all-MiniLM-L6-v2...

✓ Modelo de embeddings cargado

Ruta de documentos: c:\Users\juana\OneDrive\Documentos\UNAL - 
GITHUB\UNAL_2025_2\PLN\Sistema-multiagente-para-el-procesamiento-y-an-lisis-de-documentos-\data

...

¿Mostrar trazabilidad completa? (s/n): 

🔍 Trazabilidad del Sistema
├── Query recibida: Contrasta la agricultura orgánica y la convencional
├── QueryClassifier
│   ├── Acción: classify_intent
│   └── Detalles:
│       {
│         "query": "Contrasta la agricultura orgánica y la convencional",
│         "clasificacion": {
│           "intencion": "comparacion",
│           "confianza": 0.95,
│           "razonamiento": "El usuario solicita comparar dos métodos de agricultura, lo que implica analizar y 
│       contrastar sus diferencias y similitudes."
│         }
│       }
├── Orchestrator
│   ├── Acción: route_query
│   └── Detalles:
│       {
│         "intent": "comparacion"
│       }
├── SemanticRetriever
│   ├── Acción: retrieve_documents
│   └── Detalles:
│       {
│         "query": "Contrasta la agricultura orgánica y la convencional",
│         "num_docs_retrieved": 10,
│         "doc_sources": [
│           "c:\\Users\\juana\\OneDrive\\Documentos\\UNAL - 
│       GITHUB\\UNAL_2025_2\\PLN\\Sistema-multiagente-para-el-procesamiento-y-an-lisis-de-documentos-\\data\\Dialne
│       t-LaAgriculturaOrganicaYLaAgriculturaTradicional-5444110.pdf",
│           "c:\\Users\\juana\\OneDrive\\Documentos\\UNAL - 
│       GITHUB\\UNAL_2025_2\\PLN\\Sistema-multiagente-para-el-procesamiento-y-an-lisis-de-documentos-\\data\\Dialne
│       t-LaAgriculturaOrganicaYLaAgriculturaTradicional-5444110.pdf",
│           "c:\\Users\\juana\\OneDrive\\Documentos\\UNAL - 
│       GITHUB\\UNAL_2025_2\\PLN\\Sistema-multiagente-para-el-procesamiento-y-an-lisis-de-documentos-\\data\\Dialne
│       t-MicroemprendimientosEnAgriculturaEcologicaYMercado-7655634.pdf",
│           "c:\\Users\\juana\\OneDrive\\Documentos\\UNAL - 
│       GITHUB\\UNAL_2025_2\\PLN\\Sistema-multiagente-para-el-procesamiento-y-an-lisis-de-documentos-\\data\\Dialne
│       t-AgroecologiaYCambioClimatico-7094648.pdf",
│           "c:\\Users\\juana\\OneDrive\\Documentos\\UNAL - 
│       GITHUB\\UNAL_2025_2\\PLN\\Sistema-multiagente-para-el-procesamiento-y-an-lisis-de-documentos-\\data\\Dialne
│       t-SeguridadAlimentariaYLaAgriculturaFamiliarEnMexico-8442642.pdf",
│           "c:\\Users\\juana\\OneDrive\\Documentos\\UNAL - 
│       GITHUB\\UNAL_2025_2\\PLN\\Sistema-multiagente-para-el-procesamiento-y-an-lisis-de-documentos-\\data\\Dialne
│       t-LaAgriculturaOrganicaYLaAgriculturaTradicional-5444110.pdf",
│           "c:\\Users\\juana\\OneDrive\\Documentos\\UNAL - 
│       GITHUB\\UNAL_2025_2\\PLN\\Sistema-multiagente-para-el-procesamiento-y-an-lisis-de-documentos-\\data\\Dialne
│       t-ProcesosDeAprendizajeYModernizacionProductivaEnElA-5196238.pdf",
│           "c:\\Users\\juana\\OneDrive\\Documentos\\UNAL - 
│       GITHUB\\UNAL_2025_2\\PLN\\Sistema-multiagente-para-el-procesamiento-y-an-lisis-de-documentos-\\data\\Dialne
│       t-EfectosDeLaAgriculturaIntensivaYElCambioClimaticoS-8686684.pdf",
│           "c:\\Users\\juana\\OneDrive\\Documentos\\UNAL - 
│       GITHUB\\UNAL_2025_2\\PLN\\Sistema-multiagente-para-el-procesamiento-y-an-lisis-de-documentos-\\data\\Dialne
│       t-AgriculturaEficienteYAgriculturaEficaz-2159605.pdf",
│           "c:\\Users\\juana\\OneDrive\\Documentos\\UNAL - 
│       GITHUB\\UNAL_2025_2\\PLN\\Sistema-multiagente-para-el-procesamiento-y-an-lisis-de-documentos-\\data\\Dialne
│       t-LaAgriculturaFamiliarAgroecologicaUnaEstrategiaDeD-7537907.pdf"
│         ]
│       }
├── RAGResponseGenerator
│   ├── Acción: generate_response
│   └── Detalles:
│       {
│         "query": "Contrasta la agricultura orgánica y la convencional",
│         "intent": "comparacion",
│         "num_context_docs": 10,
│         "response_length": 2970
│       }
└── ResponseVerifier
    ├── Acción: verify_response
    └── Detalles:
        {
          "es_valida": true,
          "confianza": 0.9,
          "problemas_detectados": [],
          "sugerencias_mejora": [
            "Considerar agregar más detalles sobre las similitudes y diferencias entre la agricultura orgánica y 
        convencional",
      

================================================================================

👤 Ingresa tu consulta (o 'salir' para terminar): 

¡Hasta luego!

In [6]:
import os
from langchain_groq import ChatGroq

def build_llm_for_component(component: str):
    """
    Devuelve un LLM configurado según:
      - LLM_TIER: 'cheap' o 'full'
      - componente: 'classifier' o 'generator'

    Se usan modelos de Groq:
      - cheap -> llama-3.1-8b-instant (más barato)
      - full  -> llama-3.3-70b-versatile (más potente)

    Puedes cambiar estos modelos si quieres usar Gemini, pero la interfaz se mantiene.
    """
    tier = os.getenv("LLM_TIER", "cheap").lower()  # por defecto cheap para pruebas
    api_key = os.getenv("GROQ_API_KEY")

    if not api_key:
        raise RuntimeError("GROQ_API_KEY no está configurada en el entorno")

    if tier == "full":
        model_name = "llama-3.3-70b-versatile"
        if component == "classifier":
            max_tokens = 64
        else:  # generator
            max_tokens = 512
    else:
        # Modo económico por defecto
        model_name = "llama-3.1-8b-instant"
        if component == "classifier":
            max_tokens = 32
        else:  # generator
            max_tokens = 256

    return ChatGroq(
        model=model_name,
        api_key=api_key,
        temperature=0.0,
        max_tokens=max_tokens,
    )


In [7]:
"""
SUITE DE PRUEBAS - LANGCHAIN
"""

import json
import os
import time
import requests
from datetime import datetime
from typing import List, Dict, Any, Optional, Tuple
from rich.console import Console
from rich.table import Table
from rich.panel import Panel

console = Console()

CONFIG = {
    "GROQ_MODEL_LARGE": "llama-3.3-70b-versatile",
    "GROQ_MODEL_SMALL": "llama-3.1-8b-instant",
    "GEMINI_MODEL": "gemini-2.0-flash-exp",
    "EMBEDDING_MODEL": "all-MiniLM-L6-v2",
    "RETRIEVAL_K_SEARCH": 5,
    "RETRIEVAL_K_SUMMARY": 8,
    "RETRIEVAL_K_COMPARE": 6,
    "MAX_CONTEXT_CHARS": 3000,
    "DELAY_BETWEEN_TESTS": 10,
    "GROQ_RETRY_DELAY": 3,
    "GEMINI_RETRY_DELAY": 20,
    "GEMINI_MIN_INTERVAL": 8,
    "MAX_RETRIES": 2,
}


class GeminiClient:
    def __init__(self, api_key: str = None, model: str = None):
        self.api_key = api_key or os.getenv("GOOGLE_API_KEY")
        self.model = model or CONFIG["GEMINI_MODEL"]
        self.base_url = "https://generativelanguage.googleapis.com/v1beta"
        self.last_call_time = 0
        self.min_interval = CONFIG["GEMINI_MIN_INTERVAL"]
        self.call_count = 0
        
        if not self.api_key:
            raise ValueError("GOOGLE_API_KEY no configurada")
    
    def _wait_if_needed(self):
        elapsed = time.time() - self.last_call_time
        if elapsed < self.min_interval:
            wait = self.min_interval - elapsed
            console.print(f"[dim]⏳ Esperando {wait:.1f}s (rate limit preventivo)...[/dim]")
            time.sleep(wait)
    
    def generate(self, prompt: str, system_prompt: str = None, temperature: float = 0.3, max_tokens: int = 1024) -> Tuple[str, bool]:
        self._wait_if_needed()
        
        url = f"{self.base_url}/models/{self.model}:generateContent?key={self.api_key}"
        
        full_prompt = f"{system_prompt}\n\n{prompt}" if system_prompt else prompt
        
        payload = {
            "contents": [{"parts": [{"text": full_prompt}]}],
            "generationConfig": {
                "temperature": temperature,
                "maxOutputTokens": max_tokens,
            }
        }
        
        for attempt in range(CONFIG["MAX_RETRIES"]):
            try:
                response = requests.post(url, headers={"Content-Type": "application/json"}, json=payload, timeout=60)
                self.last_call_time = time.time()
                self.call_count += 1
                
                if response.status_code == 200:
                    data = response.json()
                    if "candidates" in data and data["candidates"]:
                        text = data["candidates"][0]["content"]["parts"][0]["text"]
                        console.print(f"[dim]✓ Gemini llamada #{self.call_count} exitosa[/dim]")
                        return text, True
                    return "Error: Respuesta vacía", False
                
                elif response.status_code == 429:
                    wait_time = CONFIG["GEMINI_RETRY_DELAY"] * (2 ** attempt)
                    console.print(f"[yellow]⚠ Gemini rate limit (intento {attempt+1}/{CONFIG['MAX_RETRIES']}), esperando {wait_time}s...[/yellow]")
                    time.sleep(wait_time)
                    continue
                else:
                    error_msg = response.text[:100]
                    console.print(f"[yellow]⚠ Gemini error {response.status_code}: {error_msg[:50]}[/yellow]")
                    if attempt < CONFIG["MAX_RETRIES"] - 1:
                        time.sleep(CONFIG["GEMINI_RETRY_DELAY"])
                        continue
                    return f"Error Gemini: {error_msg}", False
                    
            except Exception as e:
                console.print(f"[yellow]⚠ Excepción Gemini: {str(e)[:50]}[/yellow]")
                if attempt < CONFIG["MAX_RETRIES"] - 1:
                    time.sleep(5)
                    continue
                return f"Error: {str(e)[:100]}", False
        
        return "Error: Máximo de reintentos Gemini", False


class GroqWithFallback:
    """Cliente que intenta Groq primero, y usa Gemini como fallback"""
    
    def __init__(self, gemini_client: GeminiClient, model: str = None, temperature: float = 0.1):
        self.groq_api_key = os.getenv("GROQ_API_KEY")
        self.model = model or CONFIG["GROQ_MODEL_LARGE"]
        self.temperature = temperature
        self.gemini = gemini_client
        self.groq_available = False
        
        if self.groq_api_key:
            try:
                from langchain_groq import ChatGroq
                self.ChatGroq = ChatGroq
                self.groq_available = True
                console.print("[green]✓ Groq disponible[/green]")
            except Exception as e:
                console.print(f"[yellow]⚠ Error importando Groq: {e}[/yellow]")
                self.groq_available = False
        else:
            console.print("[yellow]⚠ GROQ_API_KEY no configurada, usando solo Gemini[/yellow]")
    
    def invoke(self, system_prompt: str, user_prompt: str, max_tokens: int = 256) -> Tuple[str, str]:
        """
        Intenta usar Groq. Si falla, usa Gemini.
        
        Returns:
            Tuple[str, str]: (respuesta, modelo_usado)
        """
        if self.groq_available:
            try:
                # Crear el LLM de Groq
                llm = self.ChatGroq(
                    model=self.model,
                    temperature=self.temperature,
                    groq_api_key=self.groq_api_key,
                    max_tokens=max_tokens
                )
                
                # Construir el mensaje completo
                messages = [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
                
                # Invocar directamente (sin ChatPromptTemplate para evitar errores)
                response = llm.invoke(messages)
                
                # Extraer el contenido
                if hasattr(response, 'content'):
                    result = response.content
                else:
                    result = str(response)
                
                console.print(f"[dim]✓ Groq exitoso[/dim]")
                return result, f"Groq ({self.model})"
                
            except Exception as e:
                error_str = str(e).lower()
                if "rate" in error_str or "limit" in error_str or "429" in error_str or "quota" in error_str:
                    console.print(f"[yellow]⚠ Groq rate limit, usando Gemini...[/yellow]")
                else:
                    console.print(f"[yellow]⚠ Error Groq: {str(e)[:80]}, usando Gemini...[/yellow]")
        
        # Fallback a Gemini
        response, success = self.gemini.generate(user_prompt, system_prompt, self.temperature, max_tokens)
        if success:
            return response, "Gemini (fallback)"
        else:
            return response, "ERROR"


class LocalEmbeddingsEngine:
    def __init__(self, model_name: str = None):
        from sentence_transformers import SentenceTransformer
        model_name = model_name or CONFIG["EMBEDDING_MODEL"]
        console.print(f"[yellow]⚙️ Cargando embeddings: {model_name}[/yellow]")
        self.model = SentenceTransformer(model_name)
        console.print(f"[green]✓ Embeddings cargados[/green]")
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self.model.encode(texts, convert_to_numpy=True, show_progress_bar=False).tolist()
    
    def embed_query(self, text: str) -> List[float]:
        return self.model.encode([text], convert_to_numpy=True)[0].tolist()
    
    def __call__(self, text: str) -> List[float]:
        return self.embed_query(text)


class TraceabilityLogger:
    def __init__(self):
        self.trace_log = []
        self.start_time = None
        
    def start_trace(self, query: str):
        self.start_time = datetime.now()
        self.trace_log = [{
            "timestamp": self.start_time.isoformat(),
            "event": "query_received",
            "query": query
        }]
        
    def log_event(self, agent: str, action: str, details: Dict[str, Any]):
        self.trace_log.append({
            "timestamp": datetime.now().isoformat(),
            "agent": agent,
            "action": action,
            "details": details
        })
        
    def get_trace(self) -> List[Dict]:
        return self.trace_log


class HybridIntentClassifier:
    SYSTEM_PROMPT = """Eres un clasificador de consultas experto. Clasifica la intención del usuario en:

1. "busqueda": Solicita información específica de documentos
2. "resumen": Quiere un resumen de documentos
3. "comparacion": Quiere comparar información de diferentes documentos
4. "general": Pregunta general que no requiere buscar en documentos

Responde SOLO con JSON: {"intencion": "...", "confianza": 0.95, "razonamiento": "..."}"""
    
    def __init__(self, trace_logger, groq_client: GroqWithFallback):
        self.trace_logger = trace_logger
        self.groq = groq_client
        self.cache = {}
    
    def classify(self, query: str) -> Dict[str, Any]:
        if query in self.cache:
            console.print(f"[dim]✓ Usando clasificación en caché[/dim]")
            return self.cache[query]
        
        console.print(f"[cyan]🎯 Clasificando intención...[/cyan]")
        
        response, model_used = self.groq.invoke(
            self.SYSTEM_PROMPT,
            query,
            max_tokens=100
        )
        
        try:
            response_clean = response.strip()
            if response_clean.startswith("```json"):
                response_clean = response_clean[7:]
            if response_clean.startswith("```"):
                response_clean = response_clean[3:]
            if response_clean.endswith("```"):
                response_clean = response_clean[:-3]
            
            result = json.loads(response_clean.strip())
            result["modelo_usado"] = model_used
            
            console.print(f"[green]✓ Clasificado:[/green] [bold]{result['intencion']}[/bold] [{model_used}]")
            
            self.cache[query] = result
            
            self.trace_logger.log_event(
                agent="HybridIntentClassifier",
                action="classify",
                details={"query": query[:50], "result": result}
            )
            
            return result
            
        except json.JSONDecodeError:
            console.print(f"[yellow]⚠ Error parsing, usando heurística[/yellow]")
            query_lower = query.lower()
            if any(kw in query_lower for kw in ["resume", "resumen", "sintetiza"]):
                intent = "resumen"
            elif any(kw in query_lower for kw in ["compara", "contrasta", "diferencia"]):
                intent = "comparacion"
            elif any(kw in query_lower for kw in ["qué es", "explica", "define", "cómo"]) and "documento" not in query_lower:
                intent = "general"
            else:
                intent = "busqueda"
            
            result = {"intencion": intent, "confianza": 0.6, "razonamiento": "Heurística", "modelo_usado": "heurística"}
            self.cache[query] = result
            return result


class SemanticDocumentRetriever:
    def __init__(self, vector_store, trace_logger):
        self.vector_store = vector_store
        self.trace_logger = trace_logger
    
    def retrieve(self, query: str, intent: str) -> List:
        k_map = {
            "busqueda": CONFIG["RETRIEVAL_K_SEARCH"],
            "resumen": CONFIG["RETRIEVAL_K_SUMMARY"],
            "comparacion": CONFIG["RETRIEVAL_K_COMPARE"],
        }
        k = k_map.get(intent, 5)
        
        console.print(f"[yellow]🔎 Recuperando top-{k} documentos...[/yellow]")
        
        try:
            docs = self.vector_store.similarity_search(query, k=k)
            console.print(f"[green]✓ {len(docs)} documentos recuperados[/green]")
            self.trace_logger.log_event(
                agent="SemanticDocumentRetriever",
                action="retrieve",
                details={"query": query[:50], "k": k, "retrieved": len(docs)}
            )
            return docs
        except Exception as e:
            console.print(f"[red]✗ Error: {e}[/red]")
            return []


class HybridRAGGenerator:
    SYSTEM_PROMPTS = {
        "busqueda": """Eres un asistente experto en análisis de documentos.
Responde basándote ÚNICAMENTE en el contexto proporcionado.
Sé preciso y conciso (máximo 200 palabras).""",
        
        "resumen": """Eres un experto en síntesis de información.
Genera un resumen estructurado con las ideas principales (máximo 250 palabras).""",
        
        "comparacion": """Eres un analista experto en comparación.
Compara y contrasta la información (máximo 250 palabras)."""
    }
    
    def __init__(self, trace_logger, gemini_client: GeminiClient, groq_client: GroqWithFallback):
        self.trace_logger = trace_logger
        self.gemini = gemini_client
        self.groq = groq_client
    
    def generate(self, query: str, docs: List, intent: str) -> Tuple[str, str]:
        console.print(f"[yellow]✍️ Generando respuesta RAG...[/yellow]")
        
        context_parts = []
        total_chars = 0
        for i, doc in enumerate(docs):
            content = doc.page_content[:800]
            if total_chars + len(content) > CONFIG["MAX_CONTEXT_CHARS"]:
                break
            source = doc.metadata.get('source', 'Doc')
            if '\\' in source:
                source = source.split('\\')[-1][:25]
            context_parts.append(f"[Doc {i+1}: {source}]\n{content}")
            total_chars += len(content)
        
        context = "\n\n".join(context_parts)
        system_prompt = self.SYSTEM_PROMPTS.get(intent, self.SYSTEM_PROMPTS["busqueda"])
        user_prompt = f"CONTEXTO:\n{context}\n\nPREGUNTA: {query}\n\nRESPUESTA (máximo 200 palabras):"
        
        # Intentar con Gemini primero para generación
        response, success = self.gemini.generate(user_prompt, system_prompt, temperature=0.3, max_tokens=800)
        
        if success:
            console.print(f"[green]✓ Respuesta generada [Gemini][/green]")
            self.trace_logger.log_event(
                agent="HybridRAGGenerator",
                action="generate",
                details={"intent": intent, "model": "Gemini"}
            )
            return response, "Gemini"
        
        # Fallback a Groq
        console.print(f"[yellow]⚠ Gemini falló, usando Groq...[/yellow]")
        response, model = self.groq.invoke(system_prompt, user_prompt, max_tokens=600)
        
        console.print(f"[green]✓ Respuesta generada [{model}][/green]")
        
        self.trace_logger.log_event(
            agent="HybridRAGGenerator",
            action="generate",
            details={"intent": intent, "model": model}
        )
        
        return response, model


class HybridResponseVerifier:
    def __init__(self, trace_logger, groq_client: GroqWithFallback):
        self.trace_logger = trace_logger
        self.groq = groq_client
    
    def verify(self, query: str, response: str, docs: List) -> Dict[str, Any]:
        console.print(f"[yellow]✅ Verificando...[/yellow]")
        
        if response.startswith("Error") or len(response) < 20:
            result = {"es_valida": False, "confianza": 0, "razonamiento": "Respuesta inválida", "modelo_usado": "N/A"}
        else:
            result = {"es_valida": True, "confianza": 0.8, "razonamiento": "Respuesta válida (heurística)", "modelo_usado": "heurística"}
        
        console.print(f"[green]✓ {'Válida' if result['es_valida'] else 'Inválida'}[/green]")
        
        self.trace_logger.log_event(
            agent="HybridResponseVerifier",
            action="verify",
            details=result
        )
        
        return result


class HybridOrchestrator:
    def __init__(self, classifier, retriever, generator, verifier, trace_logger, gemini_client, groq_client):
        self.classifier = classifier
        self.retriever = retriever
        self.generator = generator
        self.verifier = verifier
        self.trace_logger = trace_logger
        self.gemini = gemini_client
        self.groq = groq_client
    
    def process(self, query: str) -> Dict[str, Any]:
        self.trace_logger.start_trace(query)
        start_time = datetime.now()
        
        classification = self.classifier.classify(query)
        intent = classification["intencion"]
        
        if intent == "general":
            result = self._handle_general(query)
        else:
            result = self._handle_rag(query, intent)
        
        result["classification"] = classification
        result["processing_time"] = (datetime.now() - start_time).total_seconds()
        
        return result
    
    def _handle_general(self, query: str) -> Dict[str, Any]:
        console.print("[cyan]💬 Consulta general...[/cyan]")
        
        system = "Eres un experto. Responde de forma clara y concisa (máximo 200 palabras)."
        
        response, success = self.gemini.generate(query, system, temperature=0.7, max_tokens=500)
        
        if success:
            model_used = "Gemini"
        else:
            response, model_used = self.groq.invoke(system, query, max_tokens=400)
        
        console.print(f"[green]✓ Respuesta [{model_used}][/green]")
        
        is_valid = not response.startswith("Error")
        
        return {
            "query": query,
            "intent": "general",
            "response": response,
            "documents_used": [],
            "verification": {"es_valida": is_valid, "confianza": 1.0 if is_valid else 0},
            "model_used": model_used
        }
    
    def _handle_rag(self, query: str, intent: str) -> Dict[str, Any]:
        console.print(f"[cyan]🔄 Pipeline RAG ({intent})...[/cyan]")
        
        docs = self.retriever.retrieve(query, intent)
        
        if not docs:
            return {
                "query": query,
                "intent": intent,
                "response": "No se encontraron documentos relevantes.",
                "documents_used": [],
                "verification": {"es_valida": False, "confianza": 0},
                "model_used": "N/A"
            }
        
        response, gen_model = self.generator.generate(query, docs, intent)
        
        verification = self.verifier.verify(query, response, docs)
        
        return {
            "query": query,
            "intent": intent,
            "response": response,
            "documents_used": [doc.metadata.get('source', f'doc_{i}')[-30:] for i, doc in enumerate(docs)],
            "verification": verification,
            "model_used": gen_model
        }


class RobustTestRunner:
    def __init__(self, orchestrator):
        self.orchestrator = orchestrator
        self.results = []
    
    def run_test(self, test_id: int, query: str, expected_intent: str, description: str) -> Dict:
        console.print(f"\n{'═'*70}")
        console.print(f"[bold cyan]TEST #{test_id}[/bold cyan] │ {description}")
        console.print(f"[dim]Query: {query}[/dim]")
        console.print(f"{'═'*70}")
        
        start = datetime.now()
        
        try:
            result = self.orchestrator.process(query)
            duration = (datetime.now() - start).total_seconds()
            
            intent_match = result["intent"] == expected_intent
            is_valid = result.get("verification", {}).get("es_valida", False)
            response_ok = not result.get("response", "").startswith("Error")
            
            test_result = {
                "test_id": test_id,
                "description": description,
                "expected_intent": expected_intent,
                "actual_intent": result["intent"],
                "intent_match": intent_match,
                "response_valid": is_valid and response_ok,
                "success": intent_match and response_ok,
                "duration_seconds": duration,
                "response_preview": result.get("response", "")[:200],
                "model_used": result.get("model_used", "N/A"),
            }
            
            status = "✅" if test_result["success"] else "❌"
            console.print(f"\n{status} Intención: {result['intent']} | Válida: {is_valid and response_ok} | {duration:.1f}s | {test_result['model_used']}")
            
            if result.get("response") and response_ok:
                preview = result["response"][:300]
                console.print(Panel(preview, title="📝 Respuesta", border_style="green"))
            
        except Exception as e:
            console.print(f"[red]✗ Error: {e}[/red]")
            test_result = {
                "test_id": test_id,
                "description": description,
                "expected_intent": expected_intent,
                "success": False,
                "error": str(e)
            }
        
        self.results.append(test_result)
        return test_result
    
    def generate_report(self, output_file: str = "test_results.json"):
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(self.results, f, indent=2, ensure_ascii=False)
        
        console.print(f"\n[green]✓ Resultados guardados: {output_file}[/green]")
        
        table = Table(title="📊 Resultados")
        table.add_column("ID", width=4)
        table.add_column("Descripción", width=18)
        table.add_column("Esperada", width=11)
        table.add_column("Detectada", width=11)
        table.add_column("Estado", width=6)
        
        for r in self.results:
            status = "✅" if r.get("success") else "❌"
            table.add_row(
                str(r["test_id"]),
                r["description"][:16],
                r["expected_intent"],
                r.get("actual_intent", "ERROR"),
                status
            )
        
        console.print(table)
        
        total = len(self.results)
        if total == 0:
            return
            
        success = sum(1 for r in self.results if r.get("success"))
        
        console.print(Panel(
            f"[bold]Total:[/bold] {total}\n"
            f"[bold green]Éxito:[/bold green] {success}/{total} ({success/total*100:.1f}%)",
            title="📈 Estadísticas",
            border_style="cyan"
        ))


def run_optimized_test_suite():
    console.print(Panel(
        "[bold cyan]SUITE OPTIMIZADA - RATE LIMIT[/bold cyan]\n\n"
        "[green]✓ Delays aumentados a 10s entre pruebas[/green]\n"
        "[green]✓ Corrección error LangChain[/green]\n"
        "[green]✓ Caché de clasificaciones[/green]\n"
        "[green]✓ Verificación simplificada[/green]",
        title="🚀 Inicializando",
        border_style="green"
    ))
    
    groq_key = os.getenv("GROQ_API_KEY")
    google_key = os.getenv("GOOGLE_API_KEY")
    
    if not google_key:
        console.print("[red]ERROR: GOOGLE_API_KEY requerida[/red]")
        return
    
    console.print("[green]✓ API keys configuradas[/green]")
    
    console.print("\n[bold]Inicializando clientes...[/bold]")
    
    gemini = GeminiClient(google_key)
    groq = GroqWithFallback(gemini, CONFIG["GROQ_MODEL_LARGE"], temperature=0.1)
    
    console.print("[green]✓ Clientes inicializados[/green]")
    
    embeddings = LocalEmbeddingsEngine()
    
    trace_logger = TraceabilityLogger()
    
    from langchain_community.vectorstores import FAISS
    
    class DocumentIndexer:
        def __init__(self, embeddings_model, trace_logger):
            self.embeddings = embeddings_model
            self.trace_logger = trace_logger
            self.vector_store = None
        
        def load_index(self, path: str = "faiss_index"):
            try:
                self.vector_store = FAISS.load_local(
                    path, 
                    self.embeddings,
                    allow_dangerous_deserialization=True
                )
                console.print(f"[green]✓ Índice cargado desde: {path}[/green]")
                return self.vector_store
            except Exception as e:
                console.print(f"[red]✗ Error al cargar índice: {str(e)}[/red]")
                return None
    
    indexer = DocumentIndexer(embeddings, trace_logger)
    vector_store = indexer.load_index("faiss_index")
    
    if vector_store is None:
        console.print("[red]ERROR: No se pudo cargar el índice FAISS[/red]")
        return
    
    classifier = HybridIntentClassifier(trace_logger, groq)
    retriever = SemanticDocumentRetriever(vector_store, trace_logger)
    generator = HybridRAGGenerator(trace_logger, gemini, groq)
    verifier = HybridResponseVerifier(trace_logger, groq)
    
    orchestrator = HybridOrchestrator(classifier, retriever, generator, verifier, trace_logger, gemini, groq)
    
    runner = RobustTestRunner(orchestrator)
    
    test_cases = [
        ("¿Qué es la agricultura sostenible?", "general", "Concepto general"),
        ("Explícame qué es la rotación de cultivos", "general", "Explicación teórica"),
        ("¿Qué es la agroecología?", "general", "Definición"),
        ("¿Qué técnicas de riego se mencionan en los documentos?", "busqueda", "Búsqueda técnicas"),
        ("¿Cuáles son los principales cultivos descritos?", "busqueda", "Búsqueda fáctica"),
        ("¿Qué desafíos ambientales se mencionan?", "busqueda", "Búsqueda problemas"),
        ("Resume el contenido principal sobre control de plagas", "resumen", "Resumen plagas"),
        ("Resume las estrategias de conservación del agua", "resumen", "Resumen agua"),
        ("Compara los diferentes métodos de fertilización", "comparacion", "Comparar métodos"),
        ("Contrasta la agricultura orgánica y la convencional", "comparacion", "Contraste sistemas"),
    ]
    
    delay = CONFIG["DELAY_BETWEEN_TESTS"]
    console.print(f"\n[bold]Ejecutando {len(test_cases)} pruebas...[/bold]")
    console.print(f"[dim]Delay entre pruebas: {delay}s[/dim]\n")
    
    for idx, (query, expected, desc) in enumerate(test_cases, 1):
        runner.run_test(idx, query, expected, desc)
        if idx < len(test_cases):
            console.print(f"[dim]⏳ Esperando {delay}s antes del siguiente test...[/dim]")
            time.sleep(delay)
    
    runner.generate_report()
    
    console.print("\n[bold green]✓ Suite completada[/bold green]")


if __name__ == "__main__":
    run_optimized_test_suite()

╭─────────────────────────────────────────────── 🚀 Inicializando ────────────────────────────────────────────────╮
│ SUITE OPTIMIZADA - RATE LIMIT                                                                                   │
│                                                                                                                 │
│ ✓ Delays aumentados a 10s entre pruebas                                                                         │
│ ✓ Corrección error LangChain                                                                                    │
│ ✓ Caché de clasificaciones                                                                                      │
│ ✓ Verificación simplificada                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

✓ API keys configuradas

Inicializando clientes...

✓ Groq disponible

✓ Clientes inicializados

...

❌ Intención: comparacion | Válida: False | 178.5s | ERROR

✓ Resultados guardados: test_results.json

                          📊 Resultados                           
┏━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┓
┃ ID   ┃ Descripción        ┃ Esperada    ┃ Detectada   ┃ Estado ┃
┡━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━┩
│ 1    │ Concepto general   │ general     │ general     │ ✅     │
│ 2    │ Explicación teór   │ general     │ general     │ ✅     │
│ 3    │ Definición         │ general     │ general     │ ✅     │
│ 4    │ Búsqueda técnica   │ busqueda    │ busqueda    │ ✅     │
│ 5    │ Búsqueda fáctica   │ busqueda    │ busqueda    │ ✅     │
│ 6    │ Búsqueda problem   │ busqueda    │ busqueda    │ ✅     │
│ 7    │ Resumen plagas     │ resumen     │ resumen     │ ✅     │
│ 8    │ Resumen agua       │ resumen     │ resumen     │ ✅     │
│ 9    │ Comparar métodos   │ comparacion │ comparacion │ ✅     │
│ 10   │ Contraste sistem   │ comparacion │ comparacion │ ❌     │
└──────┴────────────────────┴─────────────┴─────────────┴────────┘

╭──────────────────────────────────────────────── 📈 Estadísticas ────────────────────────────────────────────────╮
│ Total: 10                                                                                                       │
│ Éxito: 9/10 (90.0%)                                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

✓ Suite completada